# EE-411, HomeWork 3 : Neural networks

## 1. Backpropagation with logistic loss

### 1) Predict Function

In [1]:
import numpy as np

D = 5
K = 6

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def predict(X, W):
    # X: B x D
    # W: {w1: D x K, w2: K x 1}
    # z1: B x K
    # z2: B x 1
    # yhat: B x 1
    z1 = np.dot(X, W['w1'])
    x1 = sigmoid(z1)
    z2 = np.dot(x1, W['w2'])
    yhat = sigmoid(z2)
    return z1, z2, yhat

### 2)Logistic Loss function

In [2]:

def logistic_loss(y, yhat):
    # y: B x 1
    # yhat: B x 1
    # loss: B x 1
    B = y.shape[0]
    loss_sum = np.sum(-y * np.log(yhat) - (1 - y) * np.log(1 - yhat)) / B
    return loss_sum


# testing with yhat =  y -> 0, not possible to compute log(0)
y = np.ones(10)    * 0.000000000000000
yhat = np.ones(10) * 0.000000000000000001
print(logistic_loss(y, yhat))


0.0


For $\mathbf y \simeq \mathbf{\^{y}} \simeq 0$, we get an average logistic loss of nearly 0 for the whole batch. This is expected, as the expected value and the ground truth are equal. However, $0$ is not a valid value for this is undetermined for the $log(\mathbf{\^{y}})$

### 3)Stable Logistic Loss function

To get a stable logistic loss function, we'll employ $z_2$ and the activation function (sigmoid in this case), instead of directly using $\^y$. By injecting :

$$\^y = \sigma(z_2) = \frac{1}{1 + e^{-z_2}}$$

Into:

$$ \mathcal{L} = -y\cdot log\left(\^y\right) -(1-y)\cdot log\left(1 - \^y\right)$$

We get:
$$ \mathcal{L} = -y\cdot log\left( \frac{1}{1 + e^{-z_2}}\right) -(1-y)\cdot log\left(\frac{e^{-z_2}}{1 + e^{-z_2}}\right)$$

With basic manipulations, we get the following final expression for the logistic loss function:

$$ \mathcal{L} = -z_2\cdot y + log(e^{-z_2} + 1)$$

Which does not have the same issue as the normal logistic function used above, as we have a stable function here.

In [3]:
def stable_logistic_loss(y, z2):
    # y: B x 1
    # z2: B x 1
    # loss: B x 1
    B = y.shape[0]
    loss_sum = np.sum(-y * z2 + np.logaddexp(0, z2))/B
    return loss_sum


z_2 = -10E10 * np.ones(10)
y = 0 * np.ones(10)
print("For z_2 = -10E10, y = 0")
print("Normal Logistic loss function :", logistic_loss(y, sigmoid(z_2)))
print("Stable Logistic loss function :", stable_logistic_loss(y, z_2))

For z_2 = -10E10, y = 0
Normal Logistic loss function : nan
Stable Logistic loss function : 0.0


C:\Users\osour\AppData\Local\Temp\ipykernel_7256\3499368462.py:7: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
C:\Users\osour\AppData\Local\Temp\ipykernel_7256\539335239.py:6: RuntimeWarning: divide by zero encountered in log
  loss_sum = np.sum(-y * np.log(yhat) - (1 - y) * np.log(1 - yhat)) / B
C:\Users\osour\AppData\Local\Temp\ipykernel_7256\539335239.py:6: RuntimeWarning: invalid value encountered in multiply
  loss_sum = np.sum(-y * np.log(yhat) - (1 - y) * np.log(1 - yhat)) / B


### 4) Partial Derivatives of the loss with respect to the weights


Backpropagation is, simply put, a method of calculating partial derivatives by working backwards, from the output to the intermediary results, through the weights all up to the input. The chain rule is incredibly useful for this. Let's start with:

$$ \frac{\partial}{\partial w_i} \mathcal L(\vec x, y, \vec w) == \frac{\partial\mathcal L}{\partial \^y} \cdot \frac{\partial \^y}{\partial w_i^{(2)}}$$

$$ \rightarrow \frac{\partial \^y}{\partial w_i^{(2)}} = \frac{\partial}{\partial w_i^{(2)}} \left(\frac{1}{1 + e^{-\vec w_2^T \cdot \vec x^{(1)}}}\right) = \frac{\partial}{\partial w_i^{(2)}} \left(\frac{1}{1 + e^{- w_{2,i} \cdot  x_i^{(1)}}}\right) = \frac{x_i^{(1)\cdot e^{-w_i^{(2)}x_i} }}{\left( 1 + e^{-w_i^{(2)}x_i} \right)^2 }$$

$$ \frac{\partial \mathcal L}{\partial \^y}  = \frac{\partial}{\partial \^y} \left( -y\cdot log(\^y) -(1 - y)\cdot log(1-\^y) \right) = \frac{\^y -y}{y\cdot (1-\^y)}$$

Which gives us:

$$ \frac{\partial}{\partial w_i^{(2)}} \mathcal L(\vec x, y, \vec w) = \frac{\partial \mathcal L}{\partial \^y} \cdot \frac{\partial \^y}{\partial w_i^{(2)}} = \frac{\^y -y}{y\cdot (1-\^y)} \cdot \frac{x_i^{(1)\cdot e^{-w_i^{(2)}x_i} }}{\left( 1 + e^{-w_i^{(2)}x_i} \right)^2 }$$

With the same logic, we can get the partial derivative of the loss with respect to the weights of the first layer:

$$ \frac{\partial}{\partial w_i^{(1)}} \mathcal L(\vec x, y, \vec w) = \frac{\partial \mathcal L}{\partial \^y} \cdot \frac{\partial \^y}{\partial z_2} \cdot \frac{\partial z_2}{\partial w_i^{(1)}}$$

$$ \rightarrow \frac{\partial \^y}{\partial z_2} = \frac{\partial}{\partial z_2} \left(\frac{1}{1 + e^{-\vec w_2^T \cdot \vec x^{(1)}}}\right) = \frac{\partial}{\partial z_2} \left(\frac{1}{1 + e^{- \vec w_2^T \cdot \vec x^{(1)}}}\right) = \frac{e^{-z_2}}{\left( 1 + e^{-z_2} \right)^2 }$$

$$ \frac{\partial z_2}{\partial w_i^{(1)}} = \frac{\partial}{\partial w_i^{(1)}} \left(\vec w_2^T \cdot \vec x^{(1)}\right) = \frac{\partial}{\partial w_i^{(1)}} \left(w_{2,i} \cdot  x_i^{(1)}\right) = x_i^{(1)}$$

$$ \frac{\partial \mathcal L}{\partial \^y}  = \frac{\partial}{\partial \^y} \left( -y\cdot log(\^y) -(1 - y)\cdot log(1-\^y) \right) = \frac{\^y -y}{y\cdot (1-\^y)}$$

Which gives us:

$$ \frac{\partial}{\partial w_i^{(1)}} \mathcal L(\vec x, y, \vec w) = \frac{\partial \mathcal L}{\partial \^y} \cdot \frac{\partial \^y}{\partial z_2} \cdot \frac{\partial z_2}{\partial w_i^{(1)}} = \frac{\^y -y}{y\cdot (1-\^y)} \cdot \frac{e^{-z_2}}{\left( 1 + e^{-z_2} \right)^2 } \cdot x_i^{(1)}$$


With $\vec x^{(1)}$ being the input vector, $\vec w_2$ being the weights of the second layer, and $\vec w_1$ being the weights of the first layer. By replacing:

$$ \^y = \sigma \left( \vec w^{(2)T}\cdot \sigma \left( \vec w^{(1)T}\cdot \vec x^{(0)} \right) \right)$$

and 

$$ z^{(2)} = \vec w^{(2)T}\cdot \vec x^{(1)}$$ 

Where $\sigma(z)$ is the sigmoid function:

$$ \sigma(z) = \frac{1}{1 + e^{-z}}$$




### 5) Gradient Descent function

In [4]:
def logistic_loss_grad(X, y, W):
    # X: B x D
    # y: B x 1
    # W: {w1: D x K, w2: K x 1}
    # z1: B x K
    # z2: B x 1
    # yhat: B x 1
    # grad: {w1: D x K, w2: K x 1}
    B = X.shape[0]
    z1, z2, yhat = predict(X, W)
    grad = {}
    grad['w2'] = np.dot(sigmoid(z1).T, yhat - y) / B
    grad['w1'] = np.dot(X.T, np.dot(yhat - y, W['w2'].T) * sigmoid(z1) * (1 - sigmoid(z1))) / B
    return grad

# 2. Classifying FashionMNIST using neural networks

### 1) Load dataset and construct dataloader 

The data is given in the PIL (*Python Image Library*) format. We therefore need to convert it to a type readable by the Neural Network, which is why we use *ToTensor()* transform. 

In [5]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [6]:
transform = T.Compose([
    T.ToTensor(),
    # T.Normalize((0.1307,), (0.3081,))
])

# load the train dataset
train_dataset = torchvision.datasets.MNIST(
    root='./data/', 
    train=True, 
    download=True,
    transform=transform)

# load the test dataset
test_dataset = torchvision.datasets.MNIST(
    root='./data/', 
    train=False, 
    download=True,
    transform=transform)
# hyperparameters
# 50000 images for training
# 100 epochs
# 10000 images for testing
BATCH_SIZE = 2500
TEST_BATCH_SIZE = 10000
num_epochs = 20

train_dataloader = torch.utils.data.DataLoader(
    dataset=train_dataset, 
    batch_size=BATCH_SIZE,
    shuffle=True, 
    num_workers=2)


# Construct the dataloader for the testing dataset.
test_dataloader = torch.utils.data.DataLoader(
    dataset=test_dataset, 
    batch_size=TEST_BATCH_SIZE,
    shuffle=False, 
    num_workers=2)




### 2) Multilayer Perceptron 

The MLP will be a simple two hidden layer, with 100 neurons per layer, rectified linear units as activation functions, and a linear output layer. 20 epochs are used for training, using the cross-entropy loss and the following optimizers:
1. SGD with learning rate 0.01
2. SGD with momentum 0.9, learning rate 0.01 and nesterov momentum
3. Adam with learning rate 0.01
4. Adam with learning rate 1

As seen in the exercice sessions, we'll define the following functions:

1. *train()* for training the model
2. *test()* for testing the model
3. *plot()* for plotting the results
4. *predict()* for predicting the class of a given image, and prints the percentage of confidence of the model

In [7]:
class neural_network(nn.Module):
    def __init__(self):
        super().__init__()
        # fc : fully connected, 28*28 = 784 because the images are 28x28
        # input layer
        self.fc0 = nn.Linear(784, 392)
        # first hidden layer
        self.fc1 = nn.Linear(392, 196)
        # second hidden layer 
        self.fc2 = nn.Linear(196, 98)
        # output layer
        self.fc3 = nn.Linear(98, 10)
        # activation function
        self.relu = nn.ReLU()
    
    def forward(self, x : torch.Tensor) -> torch.Tensor:
        # transform the image into a vector
        batch_size = x.shape[0]
        x = x.view(batch_size, -1)
        # forward pass through the layers
        x = self.fc0(x)
        x = self.relu(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)

        return x

def train_epoch(
    model : nn.Module,
    train_loader : DataLoader,
    optimizer : torch.optim,
    device : torch.device,
    epoch : int
    ):

    running_loss = 0.0
    model = model.to(device)
    model.train()
    for batch_index, (data, target) in enumerate(train_loader):
        # move data and target to device
        data, target = data.to(device), target.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward pass
        output = model(data)
        # compute the loss
        loss = F.cross_entropy(output, target)
        # backward pass
        loss.backward()
        # update the parameters
        optimizer.step()
        # print statistics information
        running_loss += loss.item()
    return running_loss / len(train_loader.dataset)

def fit(
    model : nn.Module,
    train_loader : DataLoader,
    optimizer : torch.optim.Optimizer,
    epochs: int,
    device : torch.device
    ):
    losses = []
    for epoch in range(epochs):
        current_loss = train_epoch( model,
                                    train_loader, 
                                    optimizer, 
                                    device, 
                                    epoch)
        print(f"Epoch {epoch} loss: {current_loss}")
        losses.append(current_loss)
    return losses

def predict(
            model : nn.Module,
            test_loader : DataLoader,
            device : torch.device
            ):

    model.eval()
    correct = 0
    test_loss = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            # forward pass
            output = model(data)
            # compute the loss
            loss = F.cross_entropy(output, target)
            test_loss += loss.item()
            # get the index of the max log-probability
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f"Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)")
    return test_loss, accuracy


# create the model
model = neural_network()
# move the model to the GPU/CPU according to availability
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(DEVICE)

# create optimizers
SGD_optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
SGD_momentum_optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True)
Adam_optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
Adam2_optimizer = torch.optim.Adam(model.parameters(), lr=1)

# sanity check 
predict(model, test_dataloader, DEVICE)

Test set: Average loss: 0.0002, Accuracy: 963/10000 (10%)


(0.00023059289455413818, 9.63)

Now that everything is set up and we performed a sanity check, we can train the 20 epochs for each optimizer type.

In [8]:
# train the model
SGD_losses = fit(model, train_dataloader, SGD_optimizer, num_epochs, DEVICE)
SGD_momentum_losses = fit(model, train_dataloader, SGD_momentum_optimizer, num_epochs, DEVICE)
Adam_losses = fit(model, train_dataloader, Adam_optimizer, num_epochs, DEVICE)
Adam2_losses = fit(model, train_dataloader, Adam2_optimizer, num_epochs, DEVICE)

# test the model
predict(model, test_dataloader, DEVICE)

# plot the losses
plt.plot(SGD_losses, label="SGD")
plt.plot(SGD_momentum_losses, label="SGD with momentum")
plt.plot(Adam_losses, label="Adam")
plt.plot(Adam2_losses, label="Adam2")
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


Epoch 0 loss: 0.0009215566754341125
Epoch 1 loss: 0.0009208874424298604
Epoch 2 loss: 0.0009202144225438436
Epoch 3 loss: 0.0009195283571879069
Epoch 4 loss: 0.0009188210566838582
Epoch 5 loss: 0.0009180858413378398
Epoch 6 loss: 0.0009173190315564473
Epoch 7 loss: 0.0009165201385815938
Epoch 8 loss: 0.0009156840244928996
Epoch 9 loss: 0.0009148061037063599
Epoch 10 loss: 0.0009138805270195007
Epoch 11 loss: 0.0009129003524780274
Epoch 12 loss: 0.0009118587096532186
Epoch 13 loss: 0.0009107450604438782
Epoch 14 loss: 0.0009095487435658773
Epoch 15 loss: 0.0009082566618919373
Epoch 16 loss: 0.0009068536599477132
Epoch 17 loss: 0.0009053213198979696
Epoch 18 loss: 0.0009036383231480916
Epoch 19 loss: 0.0009017776052157084
Epoch 0 loss: 0.0008939918359120686
Epoch 1 loss: 0.0008557843724886576
Epoch 2 loss: 0.0007341987351576488
Epoch 3 loss: 0.0004932477126518885
Epoch 4 loss: 0.00031903284788131715
Epoch 5 loss: 0.00025518782834211987
Epoch 6 loss: 0.0002224420815706253
Epoch 7 loss: 0.

### 3) Convolution Neural Network